In [1]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, BayesianEstimator, BicScore

d:\Data\Apps\python\python-3.10.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("WHO_data.csv")
model = BayesianNetwork([
    ("Fever", "Diagnosis"),("Tiredness", "Diagnosis"),
    ("Dry-Cough", "Diagnosis"),("Difficulty-in-Breathing", "Diagnosis"),
    ("Sore-Throat", "Diagnosis"),("Runny-Nose", "Diagnosis")
])

In [3]:
bic = BicScore(data)
hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method=bic)

model = BayesianNetwork(best_model.edges())
model.fit(data, estimator=BayesianEstimator)

  0%|          | 9/1000000 [00:02<64:42:19,  4.29it/s] 


In [4]:
def predict_covid(symptoms, model):
    evidence = {node: val for node, val in zip(model.nodes(), symptoms) if val is not None}
    evidence_df = pd.DataFrame([evidence])
    return model.predict_probability(evidence_df)

input_symptoms = [1, 0, 0, 1,None, 1]  # Example input array of symptoms
prob_covid = predict_covid(input_symptoms, model)
prob_covid = pd.DataFrame(prob_covid)

print("Probability of having COVID-19:", prob_covid)  # Access probability using the label "1" for COVID-19

Probability of having COVID-19:    Difficulty-in-Breathing_0  Difficulty-in-Breathing_1
0                   0.999992                   0.000008
